#

리턴값은 항상

ko,en

In [12]:
from pynput.keyboard import Controller as KeyCtrl
from pynput.keyboard import Key

In [13]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import pandas as pd
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import keyboard # 키보드 입력 감지를 위한 라이브러리
import ctypes
import time
from ctypes import wintypes
import time

# Define the allowed characters (26 lowercase + 26 uppercase)
ALPHABET = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
char_to_idx = {char: idx for idx, char in enumerate(ALPHABET)}

def one_hot_encode(letter):
    """Return a 52-dim one-hot vector for a given letter."""
    vec = np.zeros(len(ALPHABET), dtype=np.float32)
    if letter in char_to_idx:
        vec[char_to_idx[letter]] = 1.0
    return vec
class LanguageClassifier(nn.Module):
    def __init__(self, input_size=52, hidden_size=128, num_layers=1):
        super(LanguageClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, lengths):
        # Pack the padded sequence
        packed = nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_out, (h_n, _) = self.lstm(packed)
        # h_n is of shape (num_layers, batch, hidden_size); we use the last layer’s hidden state
        h_last = h_n[-1]  # shape: (batch, hidden_size)
        out = self.fc(h_last)  # shape: (batch, 1)
        out = self.sigmoid(out)  # Output between 0 and 1
        return out


model = LanguageClassifier(input_size=52, hidden_size=128, num_layers=1)

try:
    state_dict = torch.load("AI/best_model.pt")
    load_info = model.load_state_dict(state_dict, strict=False) # Keep strict=False for compatibility info

    if load_info.missing_keys or load_info.unexpected_keys:
        print(f"State dict has missing or unexpected keys.")
    else:
        print("Model loaded successfully.")

except FileNotFoundError:
    print("Model file not found.")
    # Handle file not found (e.g., initialize model differently)

except Exception as e:
    print(f"Error loading model: {e}")
    # Handle other loading errors

else:
    print("Loading process complete.") # Optional success log


def predict_language(input_str):
    model.eval()
    seq = [one_hot_encode(ch) for ch in input_str if ch in char_to_idx]
    if not seq: return 0.5  # 기본값
    
    seq_tensor = torch.tensor(seq).unsqueeze(0)
    length = torch.tensor([len(seq)])
    
    with torch.no_grad():
        output = model(seq_tensor, length)
    return output.item()

Model loaded successfully.
Loading process complete.


In [14]:
#### check if this works later
# import platform
# import os
# import ctypes


# def switch_layout(target_lang):
#     if platform.system() == 'Windows':
#         # HKL_US = 0x04090409, HKL_Korean = 0x04120412
#         lang_code = 0x04120412 if target_lang == "ko" else 0x04090409
#         ctypes.windll.user32.ActivateKeyboardLayout(lang_code, 0)
#     elif platform.system() == 'Darwin':
#         src = """
#         # macOS 입력 소스 전환 코드 (Objective-C)
#         """
#         os.system(f"osascript -e '{src}'")

In [15]:
from process.KoEnMapper import conv_en2ko, conv_ko2en

def process_conversion(new_layout):
    global buffered_word, converted_positions
    
    if new_layout == "ko":
        converted = conv_en2ko(buffered_word)
    if new_layout == "en":
        converted = conv_ko2en(buffered_word)
    
    # 백스페이스 시뮬레이션
    physical_keyboard.press(Key.backspace)
    physical_keyboard.release(Key.backspace)
    
    # 변환 텍스트 입력
    physical_keyboard.type(converted)
    converted_positions.add(len(buffered_word))

In [16]:
import subprocess
import sys
import platform
from pynput import keyboard
from datetime import datetime
import time
import ctypes  # Needed for Windows API calls
from langdetect import detect, LangDetectException
import os

# OS 감지
IS_MAC = sys.platform == 'darwin'
IS_WIN = sys.platform.startswith('win')
IS_LINUX = sys.platform.startswith('linux')

print(sys.platform)

if IS_MAC:
    print("MacOS")
    def switch_language():
        # Control 키와 Space 키를 조합하여 전환
        command = """
        osascript -e 'tell application "System Events"
            key down control
            delay 0.000
            key down space
            delay 0.000
            key up space
            delay 0.000
            key up control
        end tell'
        """
        subprocess.run(command, shell=True)
    # LEGACY CODE (checks the current input source by reading the system preferences)
    def check_language_mode():
        # MacOS의 현재 입력 소스 확인
        command = "defaults read ~/Library/Preferences/com.apple.HIToolbox.plist AppleSelectedInputSources"
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        input_source_data = result.stdout
        # English (ABC) 레이아웃 체크
        if '"KeyboardLayout Name" = ABC' in input_source_data or 'KeyboardLayout Name = ABC' in input_source_data:
            return 'en'
        # Korean 레이아웃 체크
        elif '"Bundle ID" = "com.apple.inputmethod.Korean"' in input_source_data or 'Bundle ID = com.apple.inputmethod.Korean' in input_source_data:
            return 'ko'
        else:
            return 'unknown'

elif IS_WIN:

    exe_path = "CheckCurrentLanguage.exe"
    # ULONG_PTR 타입 정의
    wintypes.ULONG_PTR = wintypes.WPARAM

    # User32.dll 로드
    hllDll = ctypes.WinDLL ("User32.dll", use_last_error=True)

    # 한/영 키 Virtual Key 코드 (VK_HANGUEL)
    VK_HANGUEL = 0x15

    # INPUT 구조체 정의
    class MOUSEINPUT(ctypes.Structure):
        _fields_ = (("dx",          wintypes.LONG),
                    ("dy",          wintypes.LONG),
                    ("mouseData",   wintypes.DWORD),
                    ("dwFlags",     wintypes.DWORD),
                    ("time",        wintypes.DWORD),
                    ("dwExtraInfo", wintypes.ULONG_PTR))

    class HARDWAREINPUT(ctypes.Structure):
        _fields_ = (("uMsg",    wintypes.DWORD),
                    ("wParamL", wintypes.WORD),
                    ("wParamH", wintypes.WORD))

    class KEYBDINPUT(ctypes.Structure):
        _fields_ = (("wVk",         wintypes.WORD), # Virtual-key code
                    ("wScan",       wintypes.WORD), # Hardware scan code
                    ("dwFlags",     wintypes.DWORD),# 플래그
                    ("time",        wintypes.DWORD),# 시간
                    ("dwExtraInfo", wintypes.ULONG_PTR))# 추가 정보

    class INPUT(ctypes.Structure):
        class _INPUT(ctypes.Union):
            _fields_ = (("ki", KEYBDINPUT),
                        ("mi", MOUSEINPUT),
                        ("hi", HARDWAREINPUT))
        _anonymous_ = ("_input",) # 익명 유니온
        _fields_ = (("type",   wintypes.DWORD), # 입력 타입
                    ("_input", _INPUT))       # 유니온 구조체

    # 한/영 상태 변경 함수
    def switch_language():
        x = INPUT(type=1 ,ki=KEYBDINPUT(wVk=VK_HANGUEL)) # 키 누름 이벤트
        y = INPUT(type=1, ki=KEYBDINPUT(wVk=VK_HANGUEL,dwFlags=2)) # 키 뗌 이벤트
        hllDll.SendInput(1, ctypes.byref(x), ctypes.sizeof(x)) # 키 누름 이벤트 전송
        time.sleep(0.05) # 딜레이
        hllDll.SendInput(1, ctypes.byref(y), ctypes.sizeof(y)) # 키 뗌 이벤트 전송
    def check_language_mode():
        """
        Executes the CheckCurrentLanguage.exe and returns the language code.

        Args:
            exe_path: The full path to the CheckCurrentLanguage.exe file.

        Returns:
            "ko" for Korean, "en" for English, "unknown" for others, or None if an error occurred.
        """
        try:
            if not os.path.exists(exe_path):
                return "Error, files does not exsist"

            result = subprocess.run([exe_path], capture_output=True, text=True, check=True)
            output = result.stdout.strip()

            if "Korean input mode" in output:
                return "ko"
            elif "English (US) keyboard layout detected" in output:
                return "en"
            elif "Detected language: ko" in output:
                return "ko"
            elif "Detected language: en" in output:
                return "en"
            elif "Detected language:" in output:
                return "unknown"
            elif "No foreground window detected" in output:
                return "en"
            elif "Korean keyboard layout detected, but no IME window found." in output:
                return "en"
            elif "Korean keyboard layout detected, but Korean IME mode is inactive." in output:
                return "en"
            else:
                return "unknown"

        except (subprocess.CalledProcessError, FileNotFoundError, Exception):
            return None

elif IS_LINUX:
    def switch_language():
        # Toggle between English and Korean layouts.
        current = check_language_mode()
        if current == 'en':
            new_layout = 'kr'
        else:
            new_layout = 'us'
        command = f"setxkbmap {new_layout}"
        subprocess.run(command, shell=True)

else:
    print("Error: Unsupported OS. This script supports only macOS, Windows, and Linux.")

win32


In [17]:
def replace_buffred_word(new_word,current_layout):
    ko_new_word = new_word
    #-- backspace simulation -- 
    if current_layout == "ko":# It is needed to measure the length of the buffered word
        new_word = conv_en2ko(new_word)
        #print(new_word)
    needed_backspace = len(new_word)
    #print(needed_backspace)
    if IS_MAC:
        for _ in range(needed_backspace):
            physical_keyboard.press(Key.backspace)
            physical_keyboard.release(Key.backspace)
    if IS_WIN:
        for _ in range(needed_backspace):
            physical_keyboard.press(Key.backspace)
            physical_keyboard.release(Key.backspace)
    #-- replace the buffered word --
    if current_layout == "ko":
        print("ko New word")
        print(ko_new_word)
        replace_word = ko_new_word
        switch_layout("en")
        #print(replace_word)
    elif current_layout == "en":
        replace_word = conv_en2ko(new_word)
        switch_layout("ko")
        #print(replace_word)
    physical_keyboard.type(replace_word)
    return replace_word

In [18]:
# current_layout = "en"
# replace_buffred_word("dlrjs gksrnrdj",current_layout)
a = input("stop here")

In [19]:
check_language_mode()

'ko'

In [23]:
# 상수 및 전역 변수 설정
THRESHOLD_HIGH = 0.8  # 영어 전환 임계값
THRESHOLD_LOW = 0.1   # 한글 전환 임계값
buffered_word = ""
physical_keyboard = KeyCtrl()
converted_positions = set()
current_layout = "Other"

In [24]:
# 로그 함수 (키 이벤트 등 기록)
log_file = 'key_event_log.txt'
def log_message(message):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open(log_file, 'a', encoding='utf-8') as f:
        f.write(f"{timestamp} - {message}\n")

# 레이아웃 전환 시 호출되는 함수
def switch_layout(new_layout):
    global current_layout, buffered_word
    if new_layout == current_layout:
        return

    # 전환 전, 지금까지 입력된 내용을 변환 (예: 영어 -> 한글 또는 한글 -> 영어)
    if new_layout == "ko":
        converted_text = conv_en2ko(buffered_word)
        print(f"Buffer conversion (Eng->Kor): '{buffered_word}' -> '{converted_text}'")
    elif new_layout == "en":
        converted_text = conv_ko2en(buffered_word)
        print(f"Buffer conversion (Kor->Eng): '{buffered_word}' -> '{converted_text}'")

    # 버퍼 초기화 후 키보드 레이아웃 전환
    buffered_word = ""
    switch_language()
    current_layout = new_layout
    print(f"Keyboard layout switched to {current_layout}")

# 키보드 이벤트 처리 함수
def on_press(key):
    global buffered_word, current_layout
    try:
        # backspace 입력 시 버퍼에서 마지막 문자 삭제
        if key == keyboard.Key.backspace:
            if buffered_word:
                try:
                    buffered_word = buffered_word[:-1]
                    #print(f"Backspace pressed. Updated buffer: '{buffered_word}'")
                    return True
                except:#already empty
                    return True
            

        # 일반 문자 입력 처리
        if hasattr(key, 'char') and key.char:
            current_layout = check_language_mode() #update current langugage
            #print(key.char)
            char = key.char
            # 숫자, 알파벳, 기호 등 필요한 문자들만 처리
            if char.isalpha() or char.isdigit() or char in "!@#$%^&*()_+-=[]{}|;:'\",.<>/?`~":
                if (current_layout == "ko"):
                    char = conv_ko2en(char)#change to english all the time
                    #print(char)
                buffered_word += char
                #print(f"Key pressed: '{char}', Buffer: '{buffered_word}'")
                
                # LSTM 모델로 언어 예측
                prob = predict_language(buffered_word)
                #print(f"LSTM prediction probability: {prob:.5f} (Eng=1, Kor=0)")

                # 현재 레이아웃과 예측값에 따라 키보드 전환
                if current_layout == "en" and prob <= THRESHOLD_LOW:
                    print("Detected Korean Switching to Korean layout.")
                    print(buffered_word)
                    replace_buffred_word(buffered_word,current_layout)
                    
                    #print("switched to ko")
                elif current_layout == "ko" and prob >= THRESHOLD_HIGH:
                    print("Detected English. Switching to English layout.")
                    print(buffered_word)
                    replace_buffred_word(buffered_word,current_layout)
                    
                    #print("switched to en")
        else:# 특수키 입력 시 버퍼 초기화
            # to do add click event
            
            #print(f"Space/Enter/Else pressed. Buffer cleared: '{buffered_word}'")
            buffered_word = ""
            return True
    except Exception as e:
        print(f"Error: {str(e)}")
        
    return True
    
def on_release(key):
    # ESC 키 입력 시 프로그램 종료
    if key == keyboard.Key.esc:
        print("ESC pressed. Exiting program.")
        return False
    return True

In [25]:
with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
        listener.join()

C:\Users\sejik\AppData\Local\Temp\ipykernel_10124\3102442863.py:70: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  seq_tensor = torch.tensor(seq).unsqueeze(0)


Detected Korean Switching to Korean layout.
dlwp
Buffer conversion (Eng->Kor): 'dlwp' -> '이제'
Keyboard layout switched to ko
Detected English. Switching to English layout.
e
ko New word
e
Buffer conversion (Kor->Eng): 'e' -> 'e'
Keyboard layout switched to en
Detected Korean Switching to Korean layout.
dlwp
Buffer conversion (Eng->Kor): 'dlwp' -> '이제'
Keyboard layout switched to ko
ESC pressed. Exiting program.
